In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as BS
from datetime import datetime

import pandas as pd
import time
import sys, os
import re
import json
import random


def find_css(css_selector, browser):
    return browser.find_element(By.CSS_SELECTOR, css_selector)
def finds_css(css_selector, browser):
    return browser.find_elements(By.CSS_SELECTOR, css_selector)

def find_xpath(xpath, browser):
    return browser.find_element(By.XPATH, xpath)
def finds_xpath(xpath, browser):
    return browser.find_elements(By.XPATH, xpath)

def find_id(e_id, browser):
    return browser.find_element(By.ID, e_id)

def find_className(cn, browser):
    return browser.find_element(By.CLASS_NAME, cn)
def finds_className(cn , browser):
    return browser.find_elements(By.CLASS_NAME, cn)

def find_linktext(lt, browser):
    return browser.find_element(By.LINK_TEXT, lt)

def find_name(name, browser):
    return browser.find_element(By.NAME, name)
def finds_name(name, browser):
    return browser.find_elements(By.NAME, name)

def find_tagName(tag_name, browser):
    return browser.find_element(By.TAG_NAME, tag_name)

def finds_tagName(tag_name, browser):
    return browser.find_elements(By.TAG_NAME, tag_name)

def resource_path(relative_path):
    """ Get absolute path to resource, works for dev and for PyInstaller """
    base_path = getattr(sys, '_MEIPASS', os.path.dirname(os.path.abspath(__file__)))
    return os.path.join(base_path, relative_path)

In [2]:
def open_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--no--sandbox')
    options.add_argument('no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1080,800')
    options.add_argument('incognito')
    chrome_service = Service('chromedriver')
    chrome_service = Service(executable_path="chromedriver.exe")
    browser = webdriver.Chrome(service=chrome_service, options=options)
    
    return browser

In [3]:
def get_url(browser ,url):
    browser.get(url)
    time.sleep(1)
    find_className('temp_btn2', browser).click()

In [200]:
df = pd.read_excel('data_352.xlsx')

df = df.fillna(0)
df['사업자등록번호'] = df['사업자등록번호'].astype('str')
df['법인등록번호'] = df['법인등록번호'].astype('str')

url_list = df['url'].tolist()
tel_list = df['전화번호'].tolist()
name_list = df['담당자'].tolist()
total_number_list = df['총인원'].tolist()
cw_list = df['계약직인원'].tolist()
free_list = df['프리랜서'].tolist()
unpaid_list = df['무급근로자'].tolist()
man_ratio_list = df['남자비율'].tolist()
office_list = df['사무직비율'].tolist()
broad_list = df['방송직인원'].tolist()
recruit_list = df['채용인원(한달)'].tolist()
newbie_list = df['신입경력비율'].tolist()
resign_list = df['퇴직인원'].tolist()
reserach_list = df['연구기술직'].tolist()
develop_list = df['개발인원'].tolist()
part_list = df['담당자 부서'].tolist()
pos_list = df['담당자 직위'].tolist()
email_list = df['담당자 이메일'].tolist()
bn_list = df['사업자등록번호'].tolist()
crn_list = df['법인등록번호'].tolist()
labeling = df['회사라벨링'].tolist()

total_number_list #총인원
cw_list # 계약직인원
free_list # 프리랜서
unpaid_list # 무급근로자
rw_list = []

for t, c, f, u in zip(total_number_list, cw_list, free_list, unpaid_list):
    rw_list.append(t - (c + f + u))

df['상용종사자'] = rw_list

In [212]:
#### 상용, 무급, 일용, 기타 인원 (남녀)
r_man_list = []
r_girl_list = []
u_man_list = []
u_girl_list = []
f_man_list = []
f_girl_list = []
c_man_list = []
c_girl_list = []

for r, m, u, f, c in zip(rw_list, man_ratio_list, unpaid_list, free_list, cw_list):
    print("상용 //", "총인원 :",r,"  남자 :", round((r*m)/10), "/ ","여자 : ",r - round((r*m)/10), "/ 합계 :", round((r*m)/10)+(r - round((r*m)/10)))
    r_man_list.append(round((r*m)/10))
    r_girl_list.append(r - round((r*m)/10))
    print("무급 //", "총인원 :",u,"  남자 :", round((u*m)/10), "/ ","여자 : ",u - round((u*m)/10), "/ 합계 :", round((u*m)/10)+(u - round((u*m)/10)))
    u_man_list.append(round((u*m)/10))
    u_girl_list.append(u - round((u*m)/10))
    print("일용 //", "총인원 :",f,"  남자 :", round((f*m)/10), "/ ","여자 : ",f - round((f*m)/10), "/ 합계 :", round((f*m)/10)+(f - round((f*m)/10)))
    f_man_list.append(round((f*m)/10))
    f_girl_list.append(f - round((f*m)/10))
    print("기타 //", "총인원 :",c,"  남자 :", round((c*m)/10), "/ ","여자 : ",c - round((c*m)/10), "/ 합계 :", round((c*m)/10)+(c - round((c*m)/10)))
    c_man_list.append(round((c*m)/10))
    c_girl_list.append(c - round((c*m)/10))
    print()

    
#### 채용 및 퇴사 인원 (남녀)
nt_man_list = []
nt_girl_list = []
ct_man_list = []
ct_girl_list = []
rt_man_list = []
rt_girl_list = []

for re, n, rs, m in zip(recruit_list, newbie_list, resign_list, man_ratio_list):
    print("총채용 :", re, "/ 신입 :",n, "/ 경력 :",re-n)
    print("신입 //", "총인원 :",n,"  남자 :", round((n*m)/10), "/ ","여자 : ",n - round((n*m)/10), "/ 합계 :", round((n*m)/10)+(n - round((n*m)/10)))
    nt_man_list.append(round((n*m)/10))
    nt_girl_list.append(n - round((n*m)/10))
    print("경력 //", "총인원 :",re-n,"  남자 :", round(((re-n)*m)/10), "/ ","여자 : ",(re-n) - round(((re-n)*m)/10), "/ 합계 :", round(((re-n)*m)/10)+((re-n) - round(((re-n)*m)/10)))
    ct_man_list.append(round(((re-n)*m)/10))
    ct_girl_list.append((re-n) - round(((re-n)*m)/10))
    print("퇴사 //", "총인원 :",rs,"  남자 :", round((rs*m)/10), "/ ","여자 : ",rs - round((rs*m)/10), "/ 합계 :", round((rs*m)/10)+(rs - round((rs*m)/10)))
    rt_man_list.append(round((rs*m)/10))
    rt_girl_list.append(rs - round((rs*m)/10))
    print()
    
    
#### 직종별 인력현황 (남녀)
broad_man_list = []
broad_girl_list = []
reserach_man_list = []
reserach_girl_list = []

for b, r, m in zip(broad_list, reserach_list, man_ratio_list):
    print("방송직 //", "총인원 :",b,"  남자 :", round((b*m)/10), "/ ","여자 : ",b - round((b*m)/10), "/ 합계 :", round((b*m)/10)+(b - round((b*m)/10)))
    broad_man_list.append(round((b*m)/10))
    broad_girl_list.append(b - round((b*m)/10))
    print("연구직 //", "총인원 :",r,"  남자 :", round((r*m)/10), "/ ","여자 : ",r - round((r*m)/10), "/ 합계 :", round((r*m)/10)+(r - round((r*m)/10)))
    reserach_man_list.append(round((r*m)/10))
    reserach_girl_list.append(r - round((r*m)/10))
    print()
   

#### 회사 라벨링
industry_mapping = {
    '방송 및 무선통신장비업': 1,
    '유선통신서비스업': 2,
    '지상파방송서비스업': 3,
    '방송프로그램 공급업': 4,
    '정보인프라서비스업': 5,
    '정보매개 및 제공서비스업': 6,
    '패키지 소프트웨어 개발 및 공급업': 7,
    'IT서비스 제공업': 8,
    '정보통신공사업': 9,
    '정보통신유통업': 10,
    '유선 통신장비업': 11,
    '음향기기업': 12,
    '통신재판매업': 13,
    '게임 소프트웨어 개발 및 공급업': 14
}

labeling_list = []

for i in df['회사라벨링']:
    labeling_list.append(industry_mapping[i])

상용 // 총인원 : 780   남자 : 546 /  여자 :  234 / 합계 : 780
무급 // 총인원 : 0   남자 : 0 /  여자 :  0 / 합계 : 0
일용 // 총인원 : 0   남자 : 0 /  여자 :  0 / 합계 : 0
기타 // 총인원 : 20   남자 : 14 /  여자 :  6 / 합계 : 20

상용 // 총인원 : 23   남자 : 21 /  여자 :  2 / 합계 : 23
무급 // 총인원 : 0   남자 : 0 /  여자 :  0 / 합계 : 0
일용 // 총인원 : 0   남자 : 0 /  여자 :  0 / 합계 : 0
기타 // 총인원 : 0   남자 : 0 /  여자 :  0 / 합계 : 0

상용 // 총인원 : 338   남자 : 169 /  여자 :  169 / 합계 : 338
무급 // 총인원 : 0   남자 : 0 /  여자 :  0 / 합계 : 0
일용 // 총인원 : 0   남자 : 0 /  여자 :  0 / 합계 : 0
기타 // 총인원 : 15   남자 : 8 /  여자 :  7 / 합계 : 15

상용 // 총인원 : 187   남자 : 94 /  여자 :  93 / 합계 : 187
무급 // 총인원 : 0   남자 : 0 /  여자 :  0 / 합계 : 0
일용 // 총인원 : 0   남자 : 0 /  여자 :  0 / 합계 : 0
기타 // 총인원 : 5   남자 : 2 /  여자 :  3 / 합계 : 5

상용 // 총인원 : 1025   남자 : 512 /  여자 :  513 / 합계 : 1025
무급 // 총인원 : 0   남자 : 0 /  여자 :  0 / 합계 : 0
일용 // 총인원 : 0   남자 : 0 /  여자 :  0 / 합계 : 0
기타 // 총인원 : 20   남자 : 10 /  여자 :  10 / 합계 : 20

상용 // 총인원 : 115   남자 : 69 /  여자 :  46 / 합계 : 115
무급 // 총인원 : 0   남자 : 0 /  여자 :  0 / 합계 : 0

In [202]:
print("url_list :", len(url_list))
print("tel_list :", len(tel_list))
print("name_list :", len(name_list))
print("total_number_list :", len(total_number_list))
# print("cw_list :", len(cw_list))
# print("free_list :", len(free_list))
# print("unpaid_list :", len(unpaid_list))
# print("man_ratio_list :", len(man_ratio_list))
print("office_list :", len(office_list))
print("broad_list :", len(broad_list))
print("recruit_list :", len(recruit_list))
print("newbie_list :", len(newbie_list))
print("resign_list :", len(resign_list))
print("reserach_list :", len(reserach_list))
print("develop_list :", len(develop_list))
print("part_list :", len(part_list))
print("pos_list :", len(pos_list))
print("email_list :", len(email_list))
print("bn_list :", len(bn_list))
print("crn_list :", len(crn_list))
# print("rw_list :", len(rw_list))
print()
print("r_man_list :", len(r_man_list))
print("r_girl_list :", len(r_girl_list))
print("u_man_list :", len(u_man_list))
print("u_girl_list :", len(u_girl_list))
print("f_man_list :", len(f_man_list))
print("f_girl_list :", len(f_girl_list))
print("c_man_list :", len(c_man_list))
print("c_girl_list :", len(c_girl_list))

url_list : 350
tel_list : 350
name_list : 350
total_number_list : 350
office_list : 350
broad_list : 350
recruit_list : 350
newbie_list : 350
resign_list : 350
reserach_list : 350
develop_list : 350
part_list : 350
pos_list : 350
email_list : 350
bn_list : 350
crn_list : 350

r_man_list : 350
r_girl_list : 350
u_man_list : 350
u_girl_list : 350
f_man_list : 350
f_girl_list : 350
c_man_list : 350
c_girl_list : 350


In [203]:
browser = open_browser()

In [162]:
get_url(browser, url_list[0])

In [149]:
def first_page_field(browser, name, part, pos, tel, email, bn, crn):
    tel_data = find_name('tel', browser).get_attribute('value')
    
    if not any(tel_data):
        # 사업자 번호
        find_name('conum1', browser).send_keys(bn[:3])
        time.sleep(.3)
        find_name('conum2', browser).send_keys(bn[3:5])
        time.sleep(.3)
        find_name('conum3', browser).send_keys(bn[5:])
        time.sleep(.3)
        
        # 법인등록 번호
        if crn != '0':
            find_name('conum4', browser).send_keys(crn[:6])
            time.sleep(.3)
            find_name('conum5', browser).send_keys(crn[6:])
            time.sleep(.3)
        
        find_name('name', browser).send_keys(name)
        time.sleep(.5)
        
        if part != 0:
            find_name('part', browser).send_keys(part)
            time.sleep(.5)
        if pos != 0:
            find_name('pos', browser).send_keys(pos)
            time.sleep(.5)
            
        find_name('tel', browser).send_keys(tel)
        time.sleep(.5)
        
        if email != 0:
            find_name('email1', browser).send_keys(email.split('@')[0])
            time.sleep(.5)
            find_name('email12', browser).send_keys(email.split('@')[1])
        
    time.sleep(1)
    find_className('temp_btn2', browser).click()

In [163]:
first_page_field(browser, name_list[0], part_list[0], pos_list[0], tel_list[0], email_list[0], bn_list[0], crn_list[0])

In [164]:
def second_page_field(browser, r_man, r_girl, u_man, u_girl, f_man, f_girl, c_man, c_girl):    
    field_data = find_name('a2e1', browser).get_attribute('value')
    field_data2 = find_name('a2e2', browser).get_attribute('value')

    if not any([field_data, field_data2]):
        # 상용 종사자 (남)
        find_name('a2b1', browser).send_keys(r_man)
        time.sleep(.5)
        # 상용 종사자 (여)
        find_name('a2b2', browser).send_keys(r_girl)
        time.sleep(.5)

        # 자영업자 및 무급가족 종사자 (남)
        find_name('a2c1', browser).send_keys(u_man)
        time.sleep(.5)
        # 자영업자 및 무급가족 종사자 (여)
        find_name('a2c2', browser).send_keys(u_girl)
        time.sleep(.5)

        # 임시 및 일용근로자 (남)
        find_name('a2d1', browser).send_keys(f_man)
        time.sleep(.5)
        # 임시 및 일용근로자 (여)
        find_name('a2d2', browser).send_keys(f_girl)
        time.sleep(.5)

        # 기타 종사자 (남)
        find_name('a2e1', browser).send_keys(c_man)
        time.sleep(.5)
        find_name('a2e2', browser).send_keys(c_girl)
        time.sleep(.5)
    
    time.sleep(1)
    find_className('temp_btn2', browser).click()

In [165]:
second_page_field(browser, r_man_list[0], r_girl_list[0], u_man_list[0], u_girl_list[0], f_man_list[0], f_girl_list[0], c_man_list[0], c_girl_list[0])

In [166]:
def third_page_field(browser, nt_man, nt_girl, ct_man, ct_girl, rt_man, rt_girl):
    field_data = find_name('a3c1', browser).get_attribute('value')
    field_data1 = find_name('a3c2', browser).get_attribute('value')

    if not any([field_data, field_data1]):
        # 채용인력 (신입 - 남)
        find_name('a3a1', browser).send_keys(nt_man)
        time.sleep(.5)
        # 채용인력 (신입 - 여)
        find_name('a3a2', browser).send_keys(nt_girl)
        time.sleep(.5)

        # 채용인력 (경력 - 여)
        find_name('a3b1', browser).send_keys(ct_man)
        time.sleep(.5)
        # 채용인력 (경력 - 여)
        find_name('a3b2', browser).send_keys(ct_girl)
        time.sleep(.5)

        # 퇴직인력 (남)
        find_name('a3c1', browser).send_keys(rt_man)
        time.sleep(.5)
        # 퇴직인력 (여)
        find_name('a3c2', browser).send_keys(rt_girl)
        
    time.sleep(1)
    find_className('temp_btn2', browser).click()

In [167]:
third_page_field(browser, nt_man_list[0], nt_girl_list[0], ct_man_list[0], ct_girl_list[0], rt_man_list[0], rt_girl_list[0])

In [177]:
def fourth_page_field(browser, reserach_man, reserach_girl, broad_man, broad_girl):
    field_data = find_name('a4e1', browser).get_attribute('value')
    field_data1 = find_name('a4e2', browser).get_attribute('value')
    
    man = find_xpath('/html/body/form/center/div/table/tbody/tr/td/table/tbody/tr[2]/td[2]', browser).text
    girl = find_xpath('/html/body/form/center/div/table/tbody/tr/td/table/tbody/tr[2]/td[3]', browser).text
    man_number = man.split(' 명')[0]
    man_number = int(man_number)
    girl_number = girl.split(' 명')[0]
    girl_number = int(girl_number)

    o_and_p_man = man_number - (broad_man_list[0] + reserach_man_list[0])
    o_and_p_girl = girl_number - (broad_girl_list[0] + reserach_girl_list[0])

    office_man = round(o_and_p_man*office_list[0]/10)
    office_girl = round(o_and_p_girl*office_list[0]/10)
    
    production_man = o_and_p_man - office_man
    production_girl = o_and_p_girl - office_girl
    
    if reserach_man or reserach_girl:
        fourth_int = 1
    else:
        fourth_int = 0

    if not any([field_data, field_data1]):
        # 연구기술직 (남)
        find_name('a4b1', browser).send_keys(reserach_man)
        time.sleep(.5)
        # 연구기술직 (여)
        find_name('a4b2', browser).send_keys(reserach_girl)
        time.sleep(.5)
        
        # 사무관리작 (남)
        find_name('a4c1', browser).send_keys(office_man)
        time.sleep(.5)
        # 사무관리작 (여)
        find_name('a4c2', browser).send_keys(office_girl)
        time.sleep(.5)
        
        # 생산직 (남)
        find_name('a4d1', browser).send_keys(production_man)
        time.sleep(.5)
        # 생산직 (여)
        find_name('a4d2', browser).send_keys(production_girl)
        time.sleep(.5)
        
        # 방송직 (남)
        find_name('a4e1', browser).send_keys(broad_man)
        time.sleep(.5)
        # 방송직 (여)
        find_name('a4e2', browser).send_keys(broad_girl)
        time.sleep(.5)
        
    time.sleep(1)
    find_className('temp_btn2', browser).click()
    
    return fourth_int

In [178]:
fourth_int = fourth_page_field(browser, reserach_man_list[0], reserach_girl_list[0], broad_man_list[0], broad_girl_list[0])

In [198]:
def fiveth_page_field(browser, sw_number, ict_content_number, ict_op_number, service_number, hw_d_number, ):
    field_data = find_name('a5a8', browser).get_attribute('value')
    
    if not any(field_data):
        # SW/SI 개발․설계
        sw = find_name('a5a1', browser)
        sw.send_keys(sw_number)
        time.sleep(.4)
        
        # ICT 콘텐츠
        ict_c = find_name('a5a2', browser)
        ict_c.send_keys(ict_content_number)
        time.sleep(.4)
        
        # 시스템 운영관리
        sy = find_name('a5a3', browser)
        sy.send_keys(ict_op_number)
        time.sleep(.4)
        
        # 정보통신 서비스
        jt = find_name('a5a4', browser)
        jt.send_keys(service_number)
        time.sleep(.4)
        
        # HW 개발․설계
        hw = find_name('a5a5', browser)
        hw.send_keys(hw_d_number)
        time.sleep(.4)
        
        # HW 유지관련
        hw_y = find_name('a5a6', browser)
        hw_y.send_keys()
        time.sleep(.4)
        
        # ICT 관련교육
        ict_e = find_name('a5a7', browser)
        ict_e.send_keys()
        time.sleep(.4)
        
        # ICT 기술영업
        ict_y = find_name('a5a8', browser)
        ict_y.send_keys()
        time.sleep(.4)

In [197]:
# SW/SI 개발․설계
sw = find_name('a5a1', browser)
# ICT 콘텐츠
ict_c = find_name('a5a2', browser)
# 시스템 운영관리
sy = find_name('a5a3', browser)
# 정보통신 서비스
jt = find_name('a5a4', browser)
# HW 개발․설계
hw = find_name('a5a5', browser)
# HW 유지관련
hw_y = find_name('a5a6', browser)
# ICT 관련교육
ict_e = find_name('a5a7', browser)
# ICT 기술영업
ict_y = find_name('a5a8', browser)


sw.send_keys(sw_number)
ict_c.send_keys(ict_content_number)
sy.send_keys(ict_op_number)
jt.send_keys(service_number)
hw.send_keys(hw_d_number)
hw_y.send_keys()
ict_e.send_keys()
ict_y.send_keys()

for d in develop_list:
    if d != 0:
        print("sw 개발 :", round(d*7/10))
        if round(d*7/10) > 30:
            print("hw 개발 :", d - round(d*7/10))
            if d - round(d*7/10) > 30:
                print("hw 유지 :", round((d - round(d*7/10))*4/10))
        print()

sw 개발 : 280
hw 개발 : 120
hw 유지 : 48

sw 개발 : 4

sw 개발 : 7

sw 개발 : 4

sw 개발 : 28

sw 개발 : 35
hw 개발 : 15

sw 개발 : 6

sw 개발 : 2

sw 개발 : 4

sw 개발 : 4

sw 개발 : 7

sw 개발 : 28

sw 개발 : 3

sw 개발 : 1

sw 개발 : 6

sw 개발 : 4

sw 개발 : 10

sw 개발 : 6

sw 개발 : 4

sw 개발 : 35
hw 개발 : 15

sw 개발 : 49
hw 개발 : 21

sw 개발 : 2

sw 개발 : 4

sw 개발 : 6

sw 개발 : 1

sw 개발 : 1

sw 개발 : 7

sw 개발 : 1

sw 개발 : 5

sw 개발 : 19

sw 개발 : 21

sw 개발 : 1

sw 개발 : 1

sw 개발 : 6

sw 개발 : 21

sw 개발 : 2

sw 개발 : 3

sw 개발 : 2

sw 개발 : 3

sw 개발 : 1

sw 개발 : 4

sw 개발 : 4

sw 개발 : 1

sw 개발 : 1

sw 개발 : 7

sw 개발 : 4

sw 개발 : 1

sw 개발 : 5

sw 개발 : 1

sw 개발 : 1

sw 개발 : 5

sw 개발 : 5

sw 개발 : 1

sw 개발 : 3

sw 개발 : 2

sw 개발 : 1

sw 개발 : 1

sw 개발 : 2

sw 개발 : 2

sw 개발 : 7

sw 개발 : 2

sw 개발 : 2

sw 개발 : 1

sw 개발 : 14

sw 개발 : 3

sw 개발 : 1

sw 개발 : 1

sw 개발 : 2

sw 개발 : 8

sw 개발 : 2

sw 개발 : 1

sw 개발 : 1

sw 개발 : 4

sw 개발 : 6

sw 개발 : 5

sw 개발 : 4

sw 개발 : 1

sw 개발 : 4

sw 개발 : 2

sw 개발 : 3

sw 개발 : 2

sw 개발 : 3

sw 개발 : 7

sw 개발 : 1

sw 개발 : 